# Meta model

Najprej sem uvozil podatkovja iz openml ki ustrezajo: 
- stevila stolpcev med 100-130 
- stevilo vrtsiv med 1000-2000

Potem sem precistil podatkovja in obdrzal tistak ki primerna za primerjavo nasim podatkov.


In [158]:
import numpy as np
import pandas as pd
import openml
from openml.tasks import TaskType

import warnings

# ne izpisuje opozoril
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)



#print(podatkovja_all)

def get_id_with_taks():
    d = openml.datasets.list_datasets(number_instances='500..3000', number_features='20..150', output_format="dataframe")
    did = list(d["did"]) #id vseh podatkovij
    tid_key = {} # kjuc s katerim bom dostopal do taksov 
    tasks = openml.tasks.list_tasks(task_type=TaskType.SUPERVISED_CLASSIFICATION, output_format="dataframe")
    tid = list(tasks['tid']) #id task
    tdid = list(tasks['did']) # id podatkovij za pripadajoci task
    ID = []
    for n,i in enumerate(tdid):
        if i in did:
           tid_key[i] = tid[n]
           ID.append(i)
    return (ID, tid_key)


def precisti_podatkovja(podatkovja, did):
    podatkovja_ok = {}
    name_key = {}
    for n, podatkovje in enumerate(podatkovja):
        name_podatkovja = podatkovje.name
        target = podatkovje.default_target_attribute
        X, _, nominal, names = podatkovje.get_data()
        # Vec pogojev, da podatkovje obdrzimo:

        # ce ga nismo ze prej (tj. neke druge verzije)
        name_ok = name_podatkovja not in podatkovja_ok

        # ce ima znan target in je target en sam (in ne npr. "Spol,Starost")
        targ_ok = target is not None and "," not in target

        # ce je target nominalen (klasifikacija) in podatki nimajo nominalnih atributov
        i_target = names.index(target) if targ_ok else 0
        nomi_ok = nominal[i_target] and sum(nominal) == 1  # natanko en nominalen in to je target
        if name_ok and targ_ok and nomi_ok:
            podatkovja_ok[name_podatkovja] = n
            name_key[name_podatkovja] = did[n]

    n_vsa = len(podatkovja)
    n_ok = len(podatkovja_ok)
    print(f"Obdrzal sem {n_ok} podatkovij ({100 * n_ok / n_vsa:.1f}%)")
    return podatkovja_ok, name_key



ID, tid_key = get_id_with_taks()
podatkovja_all = openml.datasets.get_datasets(ID)
podatkovja_ok, name_key = precisti_podatkovja(podatkovja_all, ID)


Obdrzal sem 50 podatkovij (12.7%)


Potem sem izbral primerne meta znacilke s pomocjo **pymfe**.

In [ ]:
from pymfe.mfe import MFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer

warnings.simplefilter(action='ignore', category=RuntimeError)


def generate_meta_ds(data_all, data_ok):
    meta_ds = {"name": []}
    for  data_name, n in data_ok.items():
        ds = data_all[n]
        target = ds.default_target_attribute
        X, y,_,_ = ds.get_data(target=target)
        X = np.array(X)
        y = np.array([str(t) for t in y])
        print(y)
        try:
            typ = type(X[0,np.isnan(X).any(axis=0)])
        except:
            continue
        if typ == str:
            imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent').fit(X)
        else:
            imp = SimpleImputer(missing_values=np.nan, strategy='mean').fit(X)
        X = imp.transform(X)
        mfe = MFE(groups=["general", "statistical", "info-theory"])
        mfe.fit(X, y)
        att_names, att_values = mfe.extract()

        mfeM = MFE(groups=["model-based"])
        tree = DecisionTreeClassifier()
        tree.fit(X, y)
        att_namesM, att_valuesM = mfeM.extract_from_model(tree)

        att_names += att_namesM
        att_values += att_valuesM

        is_first = len(meta_ds) == 1
        meta_ds["name"].append(data_name)
        for a_name, a_value in zip(att_names, att_values):
            if is_first:
                meta_ds[a_name] = [a_value]
            else:
                meta_ds[a_name].append(a_value)
    return pd.DataFrame(data=meta_ds, index=None)


metaDS = generate_meta_ds(data_all=podatkovja_all, data_ok=podatkovja_ok)
metaDS.to_csv("meta_learning/metaDS.csv", index=False)
print(metaDS)

Nato sem se opisal podatke.csv z mfe. Potem sem izbral 3 podobna podatkovja s pomocjo k=3 najblizjih sosedov.

In [202]:
dt = pd.read_csv('podatki.csv')
X = np.array(dt.drop('y', axis=1))
Y = dt['y']
clas = Y.unique()
Y = Y.replace(clas[0], 0)
Y = Y.replace(clas[1], 1)
y = np.array(Y)

#genereramo opis podatkov z statistikami ipd
mfe = MFE(groups=["general", "statistical", "info-theory"])
mfe.fit(X, y)
attributs_names, attributs_values = mfe.extract()
mfeM = MFE(groups=["model-based"])
tree = DecisionTreeClassifier()
tree.fit(X, y)
att_namesM, att_valuesM = mfeM.extract_from_model(tree)

attributs_names += att_namesM
attributs_values += att_valuesM

dt_mfe = pd.DataFrame(data={attributs_names[i]:attributs_values[i] for i in range(len(attributs_names))}, index=[1])
np_mfe = dt_mfe.to_numpy()


dt_meta = pd.read_csv('meta_learning/metaDS.csv')
df_names = dt_meta['name'] #imena podatkov
np_meta = dt_meta.drop('name', axis=1).to_numpy()
np_meta[np.isinf(np_meta)] = np.nan

from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean').fit(np_meta)
np_meta = imp_mean.transform(np_meta)
np_mfe = imp_mean.transform(np_mfe)
#print(np.argwhere(np.isnan(np_meta)), 'nan')

#from sklearn.impute import KNNImputer
#imputer_meta = KNNImputer(n_neighbors=2, weights="uniform")
#np_meta = imputer_meta.fit_transform(np_meta)
#np_meta = np.nan_to_num(np_meta, nan=0)

from sklearn.neighbors import NearestNeighbors

neigh = NearestNeighbors(n_neighbors=3)
neigh.fit(np_meta)

#print(np.argwhere(np.isnan(np_mfe)))
sosedi = neigh.kneighbors(np_mfe, 3, return_distance=False)[0]
sosedi = np.array(df_names[sosedi])
#sosedi = df_names[sosedi]
print(sosedi)

['PieChart2' 'cardiotocography' 'wdbc']


Nato sem preveril kateri model je bil najboljsi za posamezno podatkovje od izbtanih treh s pomocjo **openml**.

In [210]:
#data_id = name_key[sosedi[0]]
#task_id = tid_key[data_id]
#print(data_id, task_id)

def get_best_model(sosedi, name_key, metric):
    '''
    Sprejme najblizje sosede, kljuc za pridobitev id podatkovij iz imen in metriko
    vrne najbolsi model glede na metrico za dane sosede
    '''
    id_key = {v: k for k, v in name_key.items()}
    tasks = openml.tasks.list_tasks(task_type=TaskType.SUPERVISED_CLASSIFICATION, output_format="dataframe")
    k_tid = {name_key[sosedi[0]]:[], name_key[sosedi[1]]:[], name_key[sosedi[2]]:[]}
    for i, iD in enumerate(tasks['did']):
        if iD in k_tid:
            k_tid[iD].append(np.array(tasks['tid'])[i])

    models = {}
    for data_id, task_ids in k_tid.items():
        for task_id in task_ids:
            evals = openml.evaluations.list_evaluations(function=metric, 
                                                        tasks=[task_id], 
                                                        output_format="dataframe")
            if evals.empty:
                continue
            else:
                ix = evals['value'].idxmax()
                flow_name = evals['flow_name'].iloc[ix]
                value = evals['value'].iloc[ix]
                name_data = id_key[data_id]
                models[name_data] = [flow_name]
                print(f"Na podatlkih {name_data} so bili izvedeni taski:\nBest model: {flow_name}\nBest AUC: {value}\n")

    return models

metric = "area_under_roc_curve"
models = get_best_model(sosedi, name_key, metric)



Na podatlkih PieChart2 so bili izvedeni taski:
Best model: weka.kf.AttributeSelection-BestFirst-CfsSubsetEval-ReplaceMissingValues-NaiveBayes(1)
Best AUC: 0.846022

Na podatlkih cardiotocography so bili izvedeni taski:
Best model: weka.kf.RandomForest(1)
Best AUC: 0.99804

Na podatlkih cardiotocography so bili izvedeni taski:
Best model: weka.kf.RandomForest(1)
Best AUC: 0.99804

Na podatlkih wdbc so bili izvedeni taski:
Best model: sklearn.pipeline.Pipeline(imputation=openmlstudy14.preprocessing.ConditionalImputer,hotencoding=sklearn.preprocessing.data.OneHotEncoder,variencethreshold=sklearn.feature_selection.variance_threshold.VarianceThreshold,classifier=sklearn.ensemble.weight_boosting.AdaBoostClassifier(base_estimator=sklearn.tree.tree.DecisionTreeClassifier))(1)
Best AUC: 0.996049363141483

Na podatlkih wdbc so bili izvedeni taski:
Best model: mlr.classif.xgboost(6)
Best AUC: 0.995851

Na podatlkih wdbc so bili izvedeni taski:
Best model: sklearn.pipeline.Pipeline(imputer=sklearn

Testeranje najboljsih modelov na podatki.csv

In [231]:



data = pd.read_csv('podatki.csv')
X = data.drop('y', axis=1)
Y = data['y']
clas = Y.unique()
Y = Y.replace(clas[0], 0)
Y = Y.replace(clas[1], 1)

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


from sklearn.naive_bayes import GaussianNB
gNB = GaussianNB().fit(X_train, y_train)

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(1).fit(X_train, y_train)



from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
adaBoost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier).fit(X_train, y_train)

import xgboost as xgb
xgBoost =xgb.XGBClassifier(random_state=1,learning_rate=0.01).fit(X_train, y_train)

pred = gNB.predict(X_test)
aucNB= roc_auc_score(pred, y_test)
print(f'gNB auc:{aucNB}')


pred = rf.predict(X_test)
aucrf = roc_auc_score(pred, y_test)
print(f'rf auc:{aucrf}')


pred = adaBoost.predict(X_test)
aucada = roc_auc_score(pred, y_test)
print(f'adaBoost auc:{aucada}')


pred = xgBoost.predict(X_test)
aucxg = roc_auc_score(pred, y_test)
print(f'xgBoost auc:{aucxg}')



TypeError: Cannot clone object. You should provide an instance of scikit-learn estimator instead of a class.

In [110]:
id_sosedi = {'id_sosed': [], 'name_sosed':[]}

uniq_name = []
df = d.reset_index()
for s in sosedi:
    for i in range(len(df)):
        #print(df['name'].iloc[i])
        if df['name'].iloc[i] == s:
            if df['name'].iloc[i].split('_')[0] not in uniq_name:
                print(ids[i], df['name'].iloc[i])
                uniq_name.append(df['name'].iloc[i].split('_')[0])
            id_sosedi['id_sosed'].append(df['did'].iloc[i])
            id_sosedi['name_sosed'].append(df['name'].iloc[i])

#print(pd.DataFrame(id_sosedi))
print(id_sosedi)

44565 philippine_seed_2_nrows_2000_nclasses_10_ncols_100_stratify_True
43895 ibm-employee-performance
44424 eye_movements_seed_1_nrows_2000_nclasses_10_ncols_100_stratify_True
44680 sylvine_seed_2_nrows_2000_nclasses_10_ncols_100_stratify_True
1466 cardiotocography
44738 robert_seed_0_nrows_2000_nclasses_10_ncols_100_stratify_True
44623 mfeat-factors_seed_0_nrows_2000_nclasses_10_ncols_100_stratify_True
44686 christine_seed_3_nrows_2000_nclasses_10_ncols_100_stratify_True
44701 Fashion-MNIST_seed_3_nrows_2000_nclasses_10_ncols_100_stratify_True
44546 gina_seed_3_nrows_2000_nclasses_10_ncols_100_stratify_True
22 mfeat-zernike
44572 ada_seed_4_nrows_2000_nclasses_10_ncols_100_stratify_True
44560 madeline_seed_2_nrows_2000_nclasses_10_ncols_100_stratify_True
36 segment
{'id_sosed': [44565, 44566, 44567, 44563, 44564, 43895, 43897, 43905, 44424, 44439, 44426, 44441, 44425, 44440, 44423, 44438, 44427, 44442, 44680, 44682, 44679, 44681, 44678, 1466, 1560, 44738, 44740, 44741, 44742, 44739, 4

1 - weka.kf.RandomForest
2 - https://www.openml.org/search?type=flow&sort=runs&id=65%2F
https://www.openml.org/search?type=flow&sort=runs&id=1196%2F

In [90]:
import openml
from openml import tasks



benchmark = openml.study.get_suite('OpenML-CC18')
print(benchmark.tasks)
cc_ids = []
for tid in benchmark.tasks:
    tsk = tasks.list_tasks(output_format="dataframe", task_id=[tid])
    tsk = np.array(tsk['did'])
    #print(tsk)
    if tsk[0] in ids:
        print(cc_ids)
        cc_ids.append(tid)
#print(tasks.list_tasks(output_format="dataframe", task_id=[315]))
#ids_task = benchmark.tasks


[3, 6, 11, 12, 14, 15, 16, 18, 22, 23, 28, 29, 31, 32, 37, 43, 45, 49, 53, 219, 2074, 2079, 3021, 3022, 3481, 3549, 3560, 3573, 3902, 3903, 3904, 3913, 3917, 3918, 7592, 9910, 9946, 9952, 9957, 9960, 9964, 9971, 9976, 9977, 9978, 9981, 9985, 10093, 10101, 14952, 14954, 14965, 14969, 14970, 125920, 125922, 146195, 146800, 146817, 146819, 146820, 146821, 146822, 146824, 146825, 167119, 167120, 167121, 167124, 167125, 167140, 167141]
[3]
[6]
[11]
[12]
[14]
[]
[15]
[16]
[14]
[18]
[22]
[14, 16]
[23]
[28]
[29]
[31]
[14, 16, 22]
[32]
[37]
[44]
[46]
[50]
[54]
[151]
[182]
[188]
[38]
[307]
[300]
[458]
[469]
[554]
[1049]
[14, 16, 22, 31]
[1050]
[14, 16, 22, 31, 3902]
[1053]
[1063]
[1067]
[14, 16, 22, 31, 3902, 3903]
[1068]
[14, 16, 22, 31, 3902, 3903, 3917]
[1590]
[4134]
[1510]
[1489]
[1494]
[14, 16, 22, 31, 3902, 3903, 3917, 3918]
[1497]
[1501]
[1480]
[1485]
[1486]
[1487]
[14, 16, 22, 31, 3902, 3903, 3917, 3918, 9957]
[1468]
[1475]
[1462]
[1464]
[4534]
[6332]
[1461]
[4538]
[1478]
[23381]
[40499]

In [104]:
for task_id in cc_ids:  # iterate over all tasks
    #tasks.list_tasks(output_format="dataframe", task_id=[task_id])
    print(task_id)

# Return benchmark results
    evalv = openml.evaluations.list_evaluations(
                function="area_under_roc_curve", 
                tasks=[task_id], 
                output_format="dataframe"
            )
    print(np.array(evalv['value']).argmax())
    print(np.array(evalv['flow_name'])[np.array(evalv['value']).argmax()])
    

14
939
sklearn.pipeline.Pipeline(imputation=openmlstudy14.preprocessing.ConditionalImputer,hotencoding=sklearn.preprocessing.data.OneHotEncoder,variencethreshold=sklearn.feature_selection.variance_threshold.VarianceThreshold,classifier=sklearn.svm.classes.SVC)(1)
16
945
sklearn.pipeline.Pipeline(imputation=openmlstudy14.preprocessing.ConditionalImputer,hotencoding=sklearn.preprocessing.data.OneHotEncoder,variencethreshold=sklearn.feature_selection.variance_threshold.VarianceThreshold,classifier=sklearn.svm.classes.SVC)(1)
22
2583
sklearn.pipeline.Pipeline(imputation=openmlstudy14.preprocessing.ConditionalImputer,hotencoding=sklearn.preprocessing.data.OneHotEncoder,variencethreshold=sklearn.feature_selection.variance_threshold.VarianceThreshold,classifier=sklearn.svm.classes.SVC)(1)
31
9766
sklearn.pipeline.Pipeline(pca=sklearn.decomposition.pca.PCA,randomforestclassifier=sklearn.ensemble.forest.RandomForestClassifier)(1)
3902
7084
weka.RotationForest_PrincipalComponents_J48(14)
3903
78

In [103]:
evalv = openml.evaluations.list_evaluations(
                function="area_under_roc_curve", 
                tasks=[14], 
                output_format="dataframe"
        )
print(np.array(evalv['value']).argmax())
print(np.array(evalv['flow_name'])[np.array(evalv['value']).argmax()])

939
sklearn.pipeline.Pipeline(imputation=openmlstudy14.preprocessing.ConditionalImputer,hotencoding=sklearn.preprocessing.data.OneHotEncoder,variencethreshold=sklearn.feature_selection.variance_threshold.VarianceThreshold,classifier=sklearn.svm.classes.SVC)(1)


In [52]:
print(d)

         did                                               name  version  \
313      313                                       spectrometer        1   
315      315                                           us_crime        1   
316      316                                          yeast_ml8        1   
588      588                                    fri_c4_1000_100        1   
610      610                                     fri_c4_500_100        1   
...      ...                                                ...      ...   
44789  44789  KDDCup09-Upselling_seed_1_nrows_2000_nclasses_...        1   
44790  44790  KDDCup09-Upselling_seed_2_nrows_2000_nclasses_...        1   
44791  44791  KDDCup09-Upselling_seed_3_nrows_2000_nclasses_...        1   
44792  44792  KDDCup09-Upselling_seed_4_nrows_2000_nclasses_...        1   
44965  44965                       geographical_origin_of_music        7   

      uploader  status format  MajorityClassSize  MaxNominalAttDistinctValues  \
313   